# Performance analysis at application-level

## Reproduce number of method calls

In [ ]:
import py2neo
import pandas as pd

# establish connection to Neo4j database with performance data
graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password = 'neo4j')

# create cypher query to get the total number of method calls
query_method_calls ="""
MATCH (:Method:Kieker)-[calls:CALLS]->(:Method:Kieker)
RETURN SUM(calls.weight) AS methodCalls
"""

# execute query and store result in a dataframe
df_method_calls = pd.DataFrame(graph.run(query_method_calls).data())

# print out the total number of method calls (2,409,688,701)
df_method_calls.head()

## Reproduce weighted dynamic dependency graph on class level

In [ ]:
# create cypher query to calculate import, export, and combined coupling degrees
# derived from the weighted dynamic dependency graph on class level
query_c_w ="""
MATCH (t:Type:Kieker)
WHERE (t)-[:DEPENDS_ON]->() OR ()-[:DEPENDS_ON]->(t)
WITH t
OPTIONAL MATCH (t)-[out:DEPENDS_ON]->()
WITH t, SUM(out.weight) AS import
OPTIONAL MATCH ()-[in:DEPENDS_ON]->(t)
WITH t, import, SUM(in.weight) as export
RETURN ROUND(AVG(export)) AS averageExport
"""

# execute query and store result in a dataframe
df_c_w = pd.DataFrame(graph.run(query_c_w).data())

# print out the average export coupling degree on class level (370,821)
df_c_w.head()

## Reproduce weighted dynamic dependency graph on package level

In [ ]:
# create cypher query to calculate import, export, and combined coupling degrees
# derived from the weighted dynamic dependency graph on package level
query_p_w ="""
MATCH (p:Package:Kieker)
WHERE (p)-[:DEPENDS_ON]->() OR ()-[:DEPENDS_ON]->(p)
WITH p
OPTIONAL MATCH (p)-[out:DEPENDS_ON]->()
WITH p, SUM(out.weight) AS import
OPTIONAL MATCH ()-[in:DEPENDS_ON]->(p)
WITH p, import, SUM(in.weight) as export
RETURN ROUND(AVG(export)) AS averageExport
"""

# execute query and store result in a dataframe
df_p_w = pd.DataFrame(graph.run(query_p_w).data())

# print out the average export coupling degree on package level (1,868,664)
df_p_w.head()